### 1. Install and Import Libraries

In [14]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [19]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### 2. Get Neighborhoods and Coordinates

In [20]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

df = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])

for cell in soup.find("table").find_all("p"):
    postalCode = cell.find("b").get_text()
    borough = ""
    neighborhood = ""
    counter = 0
    for link in cell.find_all("a"):
        if counter == 0:
            borough = link.get_text()
        else:
            if neighborhood == "":
                neighborhood = link.get_text()
            else:
                neighborhood = neighborhood + ", " + link.get_text()
        counter += 1
    
    if borough != "":
        if neighborhood == "":
            neighborhood = borough
        df = df.append({'PostalCode': postalCode, 'Borough': borough, 'Neighborhood': neighborhood}, ignore_index=True)

In [21]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")

df_coord = df_coord.rename(columns = {"Postal Code" : "PostalCode"})

print(df_coord)

    PostalCode   Latitude  Longitude
0          M1B  43.806686 -79.194353
1          M1C  43.784535 -79.160497
2          M1E  43.763573 -79.188711
3          M1G  43.770992 -79.216917
4          M1H  43.773136 -79.239476
5          M1J  43.744734 -79.239476
6          M1K  43.727929 -79.262029
7          M1L  43.711112 -79.284577
8          M1M  43.716316 -79.239476
9          M1N  43.692657 -79.264848
10         M1P  43.757410 -79.273304
11         M1R  43.750072 -79.295849
12         M1S  43.794200 -79.262029
13         M1T  43.781638 -79.304302
14         M1V  43.815252 -79.284577
15         M1W  43.799525 -79.318389
16         M1X  43.836125 -79.205636
17         M2H  43.803762 -79.363452
18         M2J  43.778517 -79.346556
19         M2K  43.786947 -79.385975
20         M2L  43.757490 -79.374714
21         M2M  43.789053 -79.408493
22         M2N  43.770120 -79.408493
23         M2P  43.752758 -79.400049
24         M2R  43.782736 -79.442259
25         M3A  43.753259 -79.329656
2

In [22]:
df = pd.merge(df, df_coord, on="PostalCode", how="left")

print(df.to_string())

    PostalCode              Borough                                       Neighborhood   Latitude  Longitude
0          M3A           North York                                          Parkwoods  43.753259 -79.329656
1          M4A           North York                                   Victoria Village  43.725882 -79.315572
2          M5A     Downtown Toronto                          Regent Park, Harbourfront  43.654260 -79.360636
3          M6A           North York                   Lawrence Manor, Lawrence Heights  43.718518 -79.464763
4          M7A         Queen's Park                      Ontario Provincial Government  43.662301 -79.389494
5          M9A            Etobicoke                                   Islington Avenue  43.667856 -79.532242
6          M1B          Scarborough                                     Malvern, Rouge  43.806686 -79.194353
7          M3B           North York                                          Don Mills  43.745906 -79.352188
8          M4B     

### 3. Setting the Scene: Toronto

In [23]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="coursera_hhu")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(toronto_data.to_string())

   PostalCode           Borough                                       Neighborhood   Latitude  Longitude
0         M5A  Downtown Toronto                          Regent Park, Harbourfront  43.654260 -79.360636
1         M5B  Downtown Toronto                           Garden District, Ryerson  43.657162 -79.378937
2         M5C  Downtown Toronto                                     St. James Town  43.651494 -79.375418
3         M5E  Downtown Toronto                                   Downtown Toronto  43.644771 -79.373306
4         M5G  Downtown Toronto                                         Bay Street  43.657952 -79.387383
5         M6G  Downtown Toronto                                   Downtown Toronto  43.669542 -79.422564
6         M5H  Downtown Toronto                                     Richmond, King  43.650571 -79.384568
7         M5J  Downtown Toronto       Harbourfront, Union Station, Toronto Islands  43.640816 -79.381752
8         M5K  Downtown Toronto           Toronto Domin

#### 3.1 Only Boroughs Containing "Toronto"

In [29]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 3.2 Retrieving Venues

In [30]:
CLIENT_ID = 'C30IS2UE2KULOT4PMMQ24TZQWFKP0Y4PLPKI4NNQ2X2CBZIT' # your Foursquare ID
CLIENT_SECRET = 'OYIZN0VZFPOKH4G3SFHUHL2AIKPZRSR4CPV4X5HW4F5NTN1Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C30IS2UE2KULOT4PMMQ24TZQWFKP0Y4PLPKI4NNQ2X2CBZIT
CLIENT_SECRET:OYIZN0VZFPOKH4G3SFHUHL2AIKPZRSR4CPV4X5HW4F5NTN1Q


In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
LIMIT = 100

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Downtown Toronto
Bay Street
Downtown Toronto
Richmond, King
Harbourfront, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
North Toronto
University of Toronto
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport
Rosedale
Downtown Toronto
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [36]:
print(toronto_venues.shape)
toronto_venues.head()

(1298, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bay Street,86,86,86,86,86,86
"CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport",17,17,17,17,17,17
Church and Wellesley,90,90,90,90,90,90
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Downtown Toronto,167,167,167,167,167,167
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront, Union Station, Toronto Islands",100,100,100,100,100,100
"Kensington Market, Chinatown, Grange Park",87,87,87,87,87,87


In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'] , True) 

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [67]:
toronto_onehot = toronto_onehot.loc[:,~toronto_onehot.columns.duplicated()]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.00,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,0.011628,0.000000,0.00,0.000000,0.000000,0.023256,0.00,0.034884,0.000000,0.011628,0.000000,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.162791,0.000000,0.000000,0.00,0.00,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.023256,0.011628,0.011628,0.011628,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.011628,0.011628,0.000000,0.000000,0.011628,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.011628,0.000

In [68]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bay Street----
                       venue  freq
0                Coffee Shop  0.16
1         Italian Restaurant  0.05
2             Sandwich Place  0.05
3  Middle Eastern Restaurant  0.03
4               Burger Joint  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4          Boutique  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2              Gay Bar  0.04
3           Restaurant  0.03
4     Sushi Restaurant  0.03


----Commerce Court, Victoria Hotel----
                 venue  freq
0          Coffee Shop  0.11
1           Restaurant  0.07
2                 Café  0.07
3                Hotel  0.05
4  American Restaurant  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.10
1              

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### 3.3 Top 10 Venues per Neighborhood

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Burger Joint,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Chinese Restaurant,Bubble Tea Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Sculpture Garden,Bar,Boat or Ferry,Coffee Shop,Harbor / Marina,Boutique
2,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Hotel,Café,Yoga Studio,Mediterranean Restaurant,Men's Store
3,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Deli / Bodega,Gastropub,Italian Restaurant,Japanese Restaurant
4,Downtown Toronto,Coffee Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Park,Japanese Restaurant,Cheese Shop,Bakery,Cocktail Bar


### 4 Clustering the Neighborhoods

In [72]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 4, 2], dtype=int32)

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Café,Theater,Restaurant,Breakfast Spot,Farmers Market
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Café,Ramen Restaurant,Italian Restaurant,Pizza Place,Bookstore
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Bakery,Beer Bar,Cosmetics Shop,Diner,Clothing Store
3,M5E,Downtown Toronto,Downtown Toronto,43.644771,-79.373306,1,Coffee Shop,Café,Restaurant,Seafood Restaurant,Beer Bar,Park,Japanese Restaurant,Cheese Shop,Bakery,Cocktail Bar
4,M5G,Downtown Toronto,Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Italian Restaurant,Burger Joint,Ice Cream Shop,Japanese Restaurant,Middle Eastern Restaurant,Gym / Fitness Center,Chinese Restaurant,Bubble Tea Shop


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters